In [1]:
from datasets import load_dataset

/media/shagoto/Codes/2tier-test/env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
data = load_dataset("afschowdhury/mujib-dataset",split="train")

Using custom data configuration afschowdhury--mujib-dataset-ef5fe57ee22a1c28
Found cached dataset json (/home/shagoto/.cache/huggingface/datasets/afschowdhury___json/afschowdhury--mujib-dataset-ef5fe57ee22a1c28/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


In [6]:
contexts = [c['context'] for c in data]

In [7]:
contexts

[' ১৯৭০ সালের ১২ নভেম্বর পূর্ব-বাংলায় ভয়াবহ ঝড় এবং জলোচ্ছ্বাসে ১০ লাখ মানুষ মারা যায়। বঙ্গবন্ধু তার নির্বাচনি প্রচারণা স্থগিত করে ত্রাণ কাজে ঝাঁপিয়ে পড়েন।',
 "১৯৫৩ সালের নভেম্বর মাসে তৎকালীন প্রধান বিরোধী দল 'পূর্ব পাকিস্তান আওয়ামী মুসলিম লীগ' এর অধিবেশনে দলের সাধারণ সম্পাদক নির্বাচিত হন বঙ্গবন্ধু শেখ মুজিবুর রহমান। ৫ ডিসেম্বর আওয়ামী লীগের নেতৃত্বে অন্যান্য বিরোধী দলকে সঙ্গে নিয়ে গঠিত হয় যুক্তফ্রন্ট। দেশজুড়ে প্রচারণা শুরু করেন বঙ্গবন্ধু।",
 '১৯৫৪  সালের মার্চ মাসে প্রদেশিক পরিষদের সাধারণ নির্বাচনে ২৩৭টি মুসলিম আসনের মধ্যে তৎকালীন আওয়ামী মুসলিম লীগ নেতৃত্বাধীন যুক্তফ্রন্ট ২২৩টি আসনে বিজয়ী হয়। আওয়ামী লীগ বিজয়ী হয় ১৪৫টি আসনে। পাকিস্তান মুসলিম লীগের ধর্মব্যবসায়ীদের প্রত্যাখান করে বাংলার জনগণ। মাত্র ৯টি আসন পেয়ে ধুয়ে মুছে সাফ হয়ে যায় মুসলিম লীগ। বঙ্গবন্ধু মুসলিম লীগের প্রভাবশালী এক নেতাকে ১৩ হাজার ভোটের বিশাল ব্যবধানে পরাজিত করেন। দেশজুড়ে প্রচার-প্রচারণা, অক্লান্ত শ্রম এবং তেজোদীপ্ত নেতৃত্বের কারণে আওয়ামী লীগের প্রধান মুখপাত্রে পরিণত হয় বঙ্গবন্ধু। জনগণের কাছে ন্যায্য অধিকার আদায় 

In [13]:
from nlp_core.utils import utils, evaluate

context_embeddings = evaluate.get_embeddings(contexts)


In [14]:
utils.save_embeddings("context_embeddings.npy",context_embeddings)

In [20]:
utils.save_list("contexts.bin",contexts)

In [15]:
from sentence_transformers import SentenceTransformer
import numpy as np
import torch
from sentence_transformers.util import semantic_search
import pickle

from transformers import  pipeline

qa_model = pipeline("question-answering",model = "afschowdhury/afs-qa",tokenizer="afschowdhury/afs-qa")

def context_query(query):
  q_emb = evaluate.get_embeddings(query)

  query_embeddings = torch.FloatTensor(q_emb)
  hits = semantic_search(query_embeddings, context_embeddings, top_k = 7)
  print(f"User's Question: {query}")
  print("Most Similar Context Found: ")
  for i in range(len(hits[0])):
    print(f"Context {i}:" ,contexts[hits[0][i]["corpus_id"]] )
    # for item in tests:
    #   if item['question'] == questions[hits[0][i]["corpus_id"]]:
    context = contexts[hits[0][i]["corpus_id"]]
    qa_input = {'question': query,'context': context}
    res = qa_model(qa_input)
    print(res)

In [16]:
context_query("মুজিব কবে বিয়ে করেন ?")

User's Question: মুজিব কবে বিয়ে করেন ?
Most Similar Context Found: 
Context 0: ১৯৩৮ সালে আঠার বছর বয়সে ফজিলাতুন্নেসার সাথে শেখ মুজিবু্র রহমানের বিয়ে হয়। এঁদের সন্তানরা হলেন দুই কন্যা (শেখ হাসিনা, শেখ রেহানা) ও তিন পুত্র (শেখ কামাল, শেখ জামাল ও শেখ রাসেল)।
{'score': 0.9999749660491943, 'start': 0, 'end': 9, 'answer': '১৯৩৮ সালে'}
Context 1: ১৯২০ সালের ১৭ মার্চ গোপালগঞ্জ জেলার টুঙ্গিপাড়া গ্রামের এক বনেদি মুসলিম পরিবারে জন্মগ্রহণ করেন জাতির জনক বঙ্গবন্ধু শেখ মুজিবুর রহমান। পিতা শেখ লুৎফর রহমান ও মা সায়েরা খাতুন আদর করে তাকে খোকা বলে ডাকতেন। শেখ মুজিবুর রহমানের পূর্ব-পুরুষরা খ্যাতিমান আউলিয়া বায়েজিত বোস্তমি (রহ.) এর শিষ্য দরবেশ শেখ আউয়ালের বংশধর।
{'score': 0.9861412048339844, 'start': 0, 'end': 19, 'answer': '১৯২০ সালের ১৭ মার্চ'}
Context 2: ১৯৩৯ সালে কলকাতায় গিয়ে হোসেন শহীদ সোহরাওয়ার্দীর সঙ্গে সাক্ষাৎ করেন শেখ মুজিব। ফিরে এসে গোপালগঞ্জে মুসলিম ছাত্রলীগ এবং মুসলিম লীগ গঠন করেন। মুসলিম ছাত্রলীগের সম্পাদক হন তিনি নিজে। মুসলিম লীগের সম্পাদক আরেকজন হলেও মূল কাজ তাকেই করতে হতো। মাদারী

In [17]:
context_query("শেখ মুজিবের গুরু ছিলেন কে ?")

User's Question: শেখ মুজিবের গুরু ছিলেন কে ?
Most Similar Context Found: 
Context 0: ১৯২০ সালের ১৭ মার্চ গোপালগঞ্জ জেলার টুঙ্গিপাড়া গ্রামের এক বনেদি মুসলিম পরিবারে জন্মগ্রহণ করেন জাতির জনক বঙ্গবন্ধু শেখ মুজিবুর রহমান। পিতা শেখ লুৎফর রহমান ও মা সায়েরা খাতুন আদর করে তাকে খোকা বলে ডাকতেন। শেখ মুজিবুর রহমানের পূর্ব-পুরুষরা খ্যাতিমান আউলিয়া বায়েজিত বোস্তমি (রহ.) এর শিষ্য দরবেশ শেখ আউয়ালের বংশধর।
{'score': 0.8557701706886292, 'start': 293, 'end': 306, 'answer': ' শেখ আউয়ালের'}
Context 1: ১৯৩৯ সালে কলকাতায় গিয়ে হোসেন শহীদ সোহরাওয়ার্দীর সঙ্গে সাক্ষাৎ করেন শেখ মুজিব। ফিরে এসে গোপালগঞ্জে মুসলিম ছাত্রলীগ এবং মুসলিম লীগ গঠন করেন। মুসলিম ছাত্রলীগের সম্পাদক হন তিনি নিজে। মুসলিম লীগের সম্পাদক আরেকজন হলেও মূল কাজ তাকেই করতে হতো। মাদারীপুরেও মুসলিম ছাত্রলীগ গঠিত হয় তারই প্রচেষ্টায়। ক্রমেই গণতন্ত্রের মানসপুত্র সোহরাওয়ার্দীর সঙ্গে গুরু-শিষ্য সম্পর্ক তৈরি হয় শেখ মুজিবুর রহমানের।
{'score': 0.4517846703529358, 'start': 318, 'end': 333, 'answer': ' সোহরাওয়ার্দীর'}
Context 2: ১৯৪৬ সালে বিএ পড়ার সময়,

In [18]:
def context_query_only_ans(query):
  q_emb = evaluate.get_embeddings(query)
  respns = []

  query_embeddings = torch.FloatTensor(q_emb)
  hits = semantic_search(query_embeddings, context_embeddings, top_k = 7)
  # print(f"User's Question: {query}")
  for i in range(len(hits[0])):
    context = contexts[hits[0][i]["corpus_id"]]
    qa_input = {'question': query,'context': context}
    
    res = qa_model(qa_input)
    respns.append(res)
    
  max_score = max([ans['score'] for ans in respns])
  for res in respns:
    if max_score == res['score']:
      # print(f"{'*'*15} QA models best answer:  {'*'*15}")
      # print(res['answer'])
      return res['answer']

In [19]:
context_query_only_ans("শেখ মুজিবের গুরু ছিলেন কে ?")

User's Question: শেখ মুজিবের গুরু ছিলেন কে ?
*************** QA models best answer:  ***************
 হোসেন শহীদ সোহরাওয়ার্দীর
